# Отток клиентов

Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Цель: спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.  Для этого необходимо построить модель с предельно большим значением *F1*-меры. Нужно довести метрику до 0.59 на тестовой выборке.

В данном проекте мы решаем задачу классификации с помощью машинного обучения с учителем



Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Содержание

1  Подготовка данных

  - 1.1 Изучение и предобработка данных

  - 1.2 Кодирование, разделение на выборки данных

  - 1.3 Масштабирование признаков

2  Исследование задачи

3  Борьба с дисбалансом

  - 3.1  С помощью взвешивания классов

  - 3.2  С помощью увеличения выборки

  - 3.3  С помощью уменьшения выборки

4  Тестирование модели

## Подготовка данных

### Изучение и предобработка данных

Импортируем необходимые библиотеки и читаем данные из файла

In [ ]:
!pip install scikit-learn==1.1.3

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, recall_score
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')

Изучаем датасет

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Видим, что необходимо поправить названия колонок, перевести в змеиный регистр

In [ ]:
data = data.rename( 
 columns={ 
     'RowNumber': 'row_number', 
     'CustomerId': 'customer_id', 
     'Surname': 'surname',
     'CreditScore': 'credit_score',
     'Geography': 'geography',
     'Gender': 'gender',
     'Age': 'age',
     'Tenure': 'tenure',
     'Balance': 'balance',
     'NumOfProducts': 'num_of_products',
     'HasCrCard': 'has_cr_card',
     'IsActiveMember': 'is_active_member',
     'EstimatedSalary': 'estimated_salary',
     'Exited': 'exited'
 } 
)


Изучим общую информацию

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видим, что в одном из столбцов есть пропуски. В этой колонке содержится информация, сколько лет человек является клиентом банка. Рассмотрим её внимательнее

In [ ]:
data['tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: tenure, dtype: float64

В колонке практически совпадают среднее и медиана. Чтобы не терять данные для обучения модели, заполним пропуски медианой, как целым числом и проверим ещё раз описание столбца

In [ ]:
data['tenure'] = data['tenure'].fillna(data['tenure'].median())
data['tenure'].describe()

count    10000.00000
mean         4.99790
std          2.76001
min          0.00000
25%          3.00000
50%          5.00000
75%          7.00000
max         10.00000
Name: tenure, dtype: float64

Удалим столбцы с малоинформативными признаками, которые будут только мешать модели прогнозировать верно

In [ ]:
data = data.drop(columns=['row_number', 'customer_id', 'surname'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      10000 non-null  int64  
 1   geography         10000 non-null  object 
 2   gender            10000 non-null  object 
 3   age               10000 non-null  int64  
 4   tenure            10000 non-null  float64
 5   balance           10000 non-null  float64
 6   num_of_products   10000 non-null  int64  
 7   has_cr_card       10000 non-null  int64  
 8   is_active_member  10000 non-null  int64  
 9   estimated_salary  10000 non-null  float64
 10  exited            10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


### Кодирование, разделение на выборки данных 

Разделим наш датасет на признаки и целевой признак. Далее из них сделаем 3 выборки: для обучения модели, валидационную и тестовую. Отделим сначала 25% для валидационной выборки, а затем от оставшихся 75% обучающей отделим ещё 33% для теста, чтобы валидационная и тестовая выборки были примерно одинаковыми

In [ ]:
features = data.drop(columns=['exited'], axis=1)
target = data['exited']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.33, random_state=12345, stratify=target_train)

Оценим соотношение выборок

In [ ]:
features_train.shape[0]

5025

In [ ]:
features_valid.shape[0]

2500

In [ ]:
features_test.shape[0]

2475

Выделим категориальные признаки и закодируем в численные с помощью техники прямого кодирования, избегая дамми-ловушки

In [ ]:
ohe_features = features_train.select_dtypes(include='object').columns.to_list()

In [ ]:
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [ ]:
encoder_ohe.fit(features_train[ohe_features])

features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[ohe_features])

features_train = features_train.drop(ohe_features, axis=1)

features_valid[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid[ohe_features])

features_valid = features_valid.drop(ohe_features, axis=1)

features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[ohe_features])

features_test = features_test.drop(ohe_features, axis=1)

### Масштабирование признаков

Также видим, что в нашем датасете есть численные признаки, которые необходимо масштабировать, чтобы модель не давала предпочтения тем или иным

In [ ]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

После подготовки данных мы можем приступить к обучению моделей, изучив перед этим соотношение классов в целевом признаке

## Исследование задачи

Исследуем, присутствует ли в нашем датасете дисбаланс классов

In [ ]:
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Соотношение классов 4:1. Это можно исправить несколькими методами, но пока попробуем построить модели без учёта дисбаланса. Начнём с решающего дерева

In [ ]:
best_model = None
best_depth = 0
best_f1 = 0

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model = model
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth)      

Лучший показатель F1-меры: 0.565483476132191 Лучшая глубина дерева: 6


In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc)   

AUC-ROC: 0.8208949111867845


In [ ]:
best_model = None
best_depth = 0
best_f1 = 0
best_est = 0

for est in range(10, 101, 10):
    for depth in range(1, 16):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_model = model
            best_est = est
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth, 'Количество деревьев:', best_est)

Лучший показатель F1-меры: 0.5855421686746988 Лучшая глубина дерева: 14 Количество деревьев: 50


In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=14, n_estimators=50)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.8515653446402722


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
f1 = f1_score(target_valid, predictions)
print('F1-мера:', f1)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC метрика:', auc_roc)

F1-мера: 0.3265895953757226
AUC-ROC метрика: 0.7738329358340429


Лучшие показатели продемонстрировала модель случайного леса, в которой F1-мера перешагнула необходимый порог. Однако, мы можем улучшить результаты, сбалансировав классы целевого признака

## Борьба с дисбалансом

### С помощью взвешивания классов

Попробуем устранить дисбаланс с помощью взвешивания классов и посмотрим, как это отразится на результатах моделей

In [ ]:
best_model = None
best_depth = 0
best_f1 = 0

for depth in range(1, 16):
    model = DecisionTreeClassifier(class_weight='balanced', random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model = model
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth)

Лучший показатель F1-меры: 0.5836403831982314 Лучшая глубина дерева: 6


In [ ]:
model = DecisionTreeClassifier(class_weight='balanced', random_state=12345, max_depth=6)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.8448953493076409


In [ ]:
best_model = None
best_depth = 0
best_f1 = 0
best_est = 0

for est in range(10, 101, 10):
    for depth in range(1, 16):
        model = RandomForestClassifier(class_weight='balanced', random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_model = model
            best_est = est
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth, 'Количество деревьев:', best_est) 

Лучший показатель F1-меры: 0.6367795059469351 Лучшая глубина дерева: 8 Количество деревьев: 90


In [ ]:
model = RandomForestClassifier(class_weight='balanced', random_state=12345, max_depth=8, n_estimators=90)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.8665675303107598


In [ ]:
model = LogisticRegression(class_weight='balanced', random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
f1 = f1_score(target_valid, predictions)
print('F1-мера:', f1)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC метрика:', auc_roc)

F1-мера: 0.5020576131687242
AUC-ROC метрика: 0.7793114200542914


Хорошие результаты у модели случайного леса, точность модели увеличилась, показатель AUC-ROC также увеличился в сравнении с несбалансированной моделью

### С помощью увеличения выборки

Устраним дисбаланс с помощью увеличения выборки. Для этого напишем функцию, которая вернёт увеличенную сбалансированную выборку 

In [ ]:
def upsample(features_train, target_train, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled  

Увеличим класс "1" в 4 раза и поэкспериментируем с разными моделями

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
best_model = None
best_depth = 0
best_f1 = 0

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model = model
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth)

Лучший показатель F1-меры: 0.5830258302583026 Лучшая глубина дерева: 6


In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.8434536948685588


In [ ]:
best_model = None
best_depth = 0
best_f1 = 0
best_est = 0

for est in range(10, 101, 10):
    for depth in range(1, 16):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_upsampled, target_upsampled)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_model = model
            best_est = est
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth, 'Количество деревьев:', best_est) 

Лучший показатель F1-меры: 0.6402116402116401 Лучшая глубина дерева: 9 Количество деревьев: 30


In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=30)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.867592772584686


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_valid)
f1 = f1_score(target_valid, predictions)
print('F1-мера:', f1)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC метрика:', auc_roc)

F1-мера: 0.49966239027684
AUC-ROC метрика: 0.7793725990927741


Модель случайного леса при апсемплинг показывает пока что лучшие результаты. И F1-мера, и метрика AUC-ROC выше, чем при взвешивании классов

### С помощью уменьшения выборки

Устраним дисбаланс с помощью уменьшения выборки. Для этого напишем функцию, которая вернёт уменьшенную сбалансированную выборку

In [ ]:
def downsample(features_train, target_train, fraction):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

 Уменьшим класс "0" в 4 раза (т.е. оставим долю 0.25 от исходной обучающей выборки) и поэкспериментируем с моделями

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [ ]:
best_model = None
best_depth = 0
best_f1 = 0

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_downsampled, target_downsampled)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model = model
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth)

Лучший показатель F1-меры: 0.5504840940525587 Лучшая глубина дерева: 8


In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=8)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.7967183366406194


In [ ]:
best_model = None
best_depth = 0
best_f1 = 0
best_est = 0

for est in range(10, 101, 10):
    for depth in range(1, 16):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_downsampled, target_downsampled)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_model = model
            best_est = est
print('Лучший показатель F1-меры:', best_f1, 'Лучшая глубина дерева:', best_depth, 'Количество деревьев:', best_est) 

Лучший показатель F1-меры: 0.6172839506172839 Лучшая глубина дерева: 9 Количество деревьев: 100


In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=100)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC:', auc_roc) 

AUC-ROC: 0.8644617872765361


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_valid)
f1 = f1_score(target_valid, predictions)
print('F1-мера:', f1)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC метрика:', auc_roc)

F1-мера: 0.504054054054054
AUC-ROC метрика: 0.7799074223001541


Уменьшение выборки не дало ни одной модели ощутимого улучшения значений метрик. Для тестирования используем лучшую: модель случайного леса с **апсемплингом**:

model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=30)

model.fit(features_upsampled, target_upsampled)

## Тестирование модели

Используем для теста характеристики лучшей модели случайного леса по показателю F1-меры, полученные в пункте 3.2 

In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=30)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)
f1 = f1_score(target_test, predictions)
print('F1-мера на тестовой выборке:', f1)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC метрика на тестовой выборке:', auc_roc)

F1-мера на тестовой выборке: 0.5963636363636363
AUC-ROC метрика на тестовой выборке: 0.8512700023354515


А теперь попробуем добавить к нашей лучшей модели гиперпараметр взвешенных классов

In [ ]:
model = RandomForestClassifier(class_weight='balanced', random_state=12345, max_depth=9, n_estimators=30)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)
f1 = f1_score(target_test, predictions)
print('F1-мера на тестовой выборке:', f1)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC метрика на тестовой выборке:', auc_roc)

F1-мера на тестовой выборке: 0.609090909090909
AUC-ROC метрика на тестовой выборке: 0.8526591932223591


Результат стал ещё лучше! Оставляем эту модель как эталонную в рамках данного проекта

Проверим эталонную модель на адекватность. За константную модель возьмём такую, при которой все прогнозы положительные, т.е. банк решил всем клиентам раздать бонусы, чтобы удержать их. Оценим F1-меру

In [ ]:
dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(features_upsampled, target_upsampled)
f1 = f1_score(target_test, dummy_model.predict(features_test))
print('F1-мера на константной модели:', f1)

F1-мера на константной модели: 0.338368580060423


Низкое значение F1-меры на константной модели означает, что наша лучшая модель работает гораздо точнее, хоть и не идеально 

Просчитаем полноту модели

In [ ]:
recall = recall_score(target_test, predictions)
print('Recall метрика на тестовой выборке:', recall)

Recall метрика на тестовой выборке: 0.6646825396825397


Модель верно определяет 66% клиентов на отток

**Вывод:** Результат, полученный на тестовой выборке, позволяет нам сказать, что мы добились поставленной задачи и построили модель, которая будет достаточно точно прогнозировать уход клиента из банка в ближайшее время